In [ ]:
import tensorflow
import keras as k
import pandas as pd
import numpy as np
import gc
dtype_dict = {
    'P_2': 'float16',
    'D_39': 'float16',
    'B_1': 'float16',
    'B_2': 'float16',
    'R_1': 'float16',
    'S_3': 'float16',
    'D_41': 'float16',
    'B_3': 'float16',
    'D_42': 'float16',
    'D_43': 'float16',
    'D_44': 'float16',
    'B_4': 'float16',
    'D_45': 'float16',
    'B_5': 'float16',
    'R_2': 'float16',
    'D_46': 'float16',
    'D_47': 'float16',
    'D_48': 'float16',
    'D_49': 'float16',
    'B_6': 'float16',
    'B_7': 'float16',
    'B_8': 'float16',
    'D_50': 'float16',
    'D_51': 'float16',
    'B_9': 'float16',
    'R_3': 'float16',
    'D_52': 'float16',
    'P_3': 'float16',
    'B_10': 'float16',
    'D_53': 'float16',
    'S_5': 'float16',
    'B_11': 'float16',
    'S_6': 'float16',
    'D_54': 'float16',
    'R_4': 'float16',
    'S_7': 'float16',
    'B_12': 'float16',
    'S_8': 'float16',
    'D_55': 'float16',
    'D_56': 'float16',
    'B_13': 'float16',
    'R_5': 'float16',
    'D_58': 'float16',
    'S_9': 'float16',
    'B_14': 'float16',
    'D_59': 'float16',
    'D_60': 'float16',
    'D_61': 'float16',
    'B_15': 'float16',
    'S_11': 'float16',
    'D_62': 'float16',
    'D_63': 'object',
    'D_64': 'object',
    'D_65': 'float16',
    'B_16': 'float16',
    'B_17': 'float16',
    'B_18': 'float16',
    'B_19': 'float16',
    'D_66': 'float16',
    'B_20': 'float16',
    'D_68': 'float16',
    'S_12': 'float16',
    'R_6': 'float16',
    'S_13': 'float16',
    'B_21': 'float16',
    'D_69': 'float16',
    'B_22': 'float16',
    'D_70': 'float16',
    'D_71': 'float16',
    'D_72': 'float16',
    'S_15': 'float16',
    'B_23': 'float16',
    'D_73': 'float16',
    'P_4': 'float16',
    'D_74': 'float16',
    'D_75': 'float16',
    'D_76': 'float16',
    'B_24': 'float16',
    'R_7': 'float16',
    'D_77': 'float16',
    'B_25': 'float16',
    'B_26': 'float16',
    'D_78': 'float16',
    'D_79': 'float16',
    'R_8': 'float16',
    'R_9': 'float16',
    'S_16': 'float16',
    'D_80': 'float16',
    'R_10': 'float16',
    'R_11': 'float16',
    'B_27': 'float16',
    'D_81': 'float16',
    'D_82': 'float16',
    'S_17': 'float16',
    'R_12': 'float16',
    'B_28': 'float16',
    'R_13': 'float16',
    'D_83': 'float16',
    'R_14': 'float16',
    'R_15': 'float16',
    'D_84': 'float16',
    'R_16': 'float16',
    'B_29': 'float16',
    'B_30': 'float16',
    'S_18': 'float16',
    'D_86': 'float16',
    'D_87': 'float16',
    'R_17': 'float16',
    'R_18': 'float16',
    'D_88': 'float16',
    'B_31': 'int64',
    'S_19': 'float16',
    'R_19': 'float16',
    'B_32': 'float16',
    'S_20': 'float16',
    'R_20': 'float16',
    'R_21': 'float16',
    'B_33': 'float16',
    'D_89': 'float16',
    'R_22': 'float16',
    'R_23': 'float16',
    'D_91': 'float16',
    'D_92': 'float16',
    'D_93': 'float16',
    'D_94': 'float16',
    'R_24': 'float16',
    'R_25': 'float16',
    'D_96': 'float16',
    'S_22': 'float16',
    'S_23': 'float16',
    'S_24': 'float16',
    'S_25': 'float16',
    'S_26': 'float16',
    'D_102': 'float16',
    'D_103': 'float16',
    'D_104': 'float16',
    'D_105': 'float16',
    'D_106': 'float16',
    'D_107': 'float16',
    'B_36': 'float16',
    'B_37': 'float16',
    'R_26': 'float16',
    'R_27': 'float16',
    'B_38': 'float16',
    'D_108': 'float16',
    'D_109': 'float16',
    'D_110': 'float16',
    'D_111': 'float16',
    'B_39': 'float16',
    'D_112': 'float16',
    'B_40': 'float16',
    'S_27': 'float16',
    'D_113': 'float16',
    'D_114': 'float16',
    'D_115': 'float16',
    'D_116': 'float16',
    'D_117': 'float16',
    'D_118': 'float16',
    'D_119': 'float16',
    'D_120': 'float16',
    'D_121': 'float16',
    'D_122': 'float16',
    'D_123': 'float16',
    'D_124': 'float16',
    'D_125': 'float16',
    'D_126': 'float16',
    'D_127': 'float16',
    'D_128': 'float16',
    'D_129': 'float16',
    'B_41': 'float16',
    'B_42': 'float16',
    'D_130': 'float16',
    'D_131': 'float16',
    'D_132': 'float16',
    'D_133': 'float16',
    'R_28': 'float16',
    'D_134': 'float16',
    'D_135': 'float16',
    'D_136': 'float16',
    'D_137': 'float16',
    'D_138': 'float16',
    'D_139': 'float16',
    'D_140': 'float16',
    'D_141': 'float16',
    'D_142': 'float16',
    'D_143': 'float16',
    'D_144': 'float16',
    'D_145': 'float16'}
data = pd.read_csv('train_data_3,5.csv', dtype = dtype_dict)
data.drop(columns= ["Unnamed: 0"], inplace = True)
data.head()

In [ ]:
data.head()

In [ ]:
train = pd.read_csv('train_data_3,5.csv', usecols=['customer_ID'])
#train['customer_ID'] = train['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
train['customer_ID'] = train['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int64')
customers = train.drop_duplicates().sort_index().values.flatten()
print(f'There are {len(customers)} unique customers in train.')


In [ ]:
train = pd.read_csv('train_data_3,5.csv', dtype = dtype_dict)
train.drop(columns= ["Unnamed: 0"], inplace = True)
T_COLS = train.columns.tolist()
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
]

features = train.drop(["customer_ID", "S_2"], axis = 1).columns.to_list()
num_features = [col for col in features if col not in cat_features]
train['customer_ID'] = train['customer_ID'].apply(int, base = 16)
train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last',])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace=True)
train_num_agg.head(10)
print(f'There are {len(T_COLS)} train dataframe columns')
train_cat_agg = train.groupby("customer_ID")[cat_features].agg(["count", "first", "last", "nunique"])
train_cat_agg.columns = ["_".join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)
train = pd.concat([train_num_agg, train_cat_agg], axis= 1)

In [ ]:
train.head()

In [ ]:
train.loc

In [ ]:
list_train_cat_agg = train_cat_agg.columns.tolist()
list_train_cat_agg.pop(0)
list_train_cat_agg

In [ ]:
train.shape

In [ ]:
def feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None):

    # REDUCE STRING COLUMNS
    # from 64 bytes to 8 bytes, and 10 bytes to 3 bytes respectively

    # LABEL ENCODE CAT COLUMNS (and reduce to 1 byte)
    # with 0: padding, 1: nan, 2,3,4,etc: values
    d_63_map = {'CL':2, 'CO':3, 'CR':4, 'XL':5, 'XM':6, 'XZ':7}
    train['D_63_last'] = train.D_63_last.map(d_63_map).fillna(1).astype('int16')
    train['D_63_first'] = train.D_63_first.map(d_63_map).fillna(1).astype('int16')

    d_64_map = {'-1':2,'O':3, 'R':4, 'U':5}
    train['D_64_last'] = train.D_64_last.map(d_64_map).fillna(1).astype('int16')
    train['D_64_first'] = train.D_64_first.map(d_64_map).fillna(1).astype('int16')

    CATS = [list_train_cat_agg]
    OFFSETS = [2,1,2,2,3,2,3,2,2] #2 minus minimal value in full train csv
    # then 0 will be padding, 1 will be NAN, 2,3,4,etc will be values
    for c,s in zip(CATS,OFFSETS):
        train[c] = train[c] + s
        train[c] = train[c].fillna(1).astype('int16')
    CATS += ['D_63_first','D_64_last']

    # ADD NEW FEATURES HERE
    # EXAMPLE: train['feature_189'] = etc etc etc
    # EXAMPLE: train['feature_190'] = etc etc etc
    # IF CATEGORICAL, THEN ADD TO CATS WITH: CATS += ['feaure_190'] etc etc etc

    # PAD ROWS SO EACH CUSTOMER HAS 13 ROWS


    # ADD TARGETS (and reduce to 1 byte)
    if targets is not None:
        train = train.merge(targets,on='customer_ID',how='left')
        train.target = train.target.astype('int16')

    # FILL NAN
    d_63_map = {'CL':2, 'CO':3, 'CR':4, 'XL':5, 'XM':6, 'XZ':7}
    train['D_63'] = train.D_63.map(d_63_map).fillna(1).astype('int8')

    d_64_map = {'-1':2,'O':3, 'R':4, 'U':5}
    train['D_64'] = train.D_64.map(d_64_map).fillna(1).astype('int8')

    CATS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
    OFFSETS = [2,1,2,2,3,2,3,2,2] #2 minus minimal value in full train csv
    # then 0 will be padding, 1 will be NAN, 2,3,4,etc will be values
    for c,s in zip(CATS,OFFSETS):
        train[c] = train[c] + s
        train[c] = train[c].fillna(1).astype('int8')
    CATS += ['D_63','D_64']

    # SORT BY CUSTOMER THEN DATE
    train = train.sort_values(['customer_ID','year','month','day']).reset_index(drop=True)
    train = train.drop(['year','month','day'],axis=1)

    # REARRANGE COLUMNS WITH 11 CATS FIRST
    COLS = list(train.columns[1:])
    COLS = ['customer_ID'] + CATS + [c for c in COLS if c not in CATS]
    train = train[COLS]

    return train

In [ ]:
train = feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None)
train.head()